# Calculate center of mass of a VASP file

In [1]:
from ase.io import read, write

struct=read('POSCAR')

Center = struct.get_center_of_mass()
cellpar = struct.cell.cellpar()
print(Center[0]/cellpar[0])
print(Center[1]/cellpar[1])
print(Center[2]/cellpar[2])

0.5002027558299786
0.5220572348200845
0.5161193059565831


# Convert a VASP file to  CIF or XYZ 

In [ ]:
from ase.io import read, write
struct=read('final.vasp')
write('OSCAR1.cif',struct)
write('OSCAR1.xyz',struct)

In [4]:
from ase.io import read, write
struct=read('s1ptmsp1.xyz')
# cell = struct.get_cell()
# cell.round(3)
write('try0.cif',struct)
# write('try.pdb',struct)

# Add lattice parameters to a XYZ data file and having output of CIF

In [7]:
from ase.io import read, write
from ase import Atoms
import numpy as np

# Read the XYZ file
atoms = read('s2ptmsp1.xyz')

# Define lattice vectors for non-cubic lattice
# Replace a_length, b_length, c_length with your lattice lengths for x, y, z axes
a_length = 115.593  # Length along x-axis in Angstroms
b_length = 57.7965  # Length along y-axis in Angstroms
c_length = 57.7965  # Length along z-axis in Angstroms

lattice_vectors = np.array([[a_length, 0, 0],
                            [0, b_length, 0],
                            [0, 0, c_length]])

# Set the lattice (cell)
atoms.set_cell(lattice_vectors)

# Ensure periodic boundary conditions are set if it's a bulk structure
atoms.set_pbc([True, True, True])

# Write to a VASP POSCAR file
write('try3.cif', atoms)


# Splitting a XYZ trajectory (LAMMPS output) to sub XYZ files

In [18]:
from ase.io import read, write
import numpy as np



def extract_lattice_lengths(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    lattice_lengths = []
    read_bounds = False

    for line in lines:
        if line.startswith("ITEM: BOX BOUNDS"):
            read_bounds = True
            bounds = []
        elif read_bounds:
            if len(bounds) < 3:
                bounds.append(line.split())
            else:
                # Calculate lengths and reset for next structure
                lengths = [float(b[1]) - float(b[0]) for b in bounds]
                lattice_lengths.append(lengths)
                read_bounds = False

    return lattice_lengths

def split_xyz_file(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    structure = []
    file_count = 0

    for line in lines:
        if line.startswith("38010"):
            if structure:
                with open(f's{file_count}ptmsp2.xyz', 'w') as out_file:
                    out_file.writelines(structure)
                file_count += 1
                structure = []
                structure.append(line)
        else:
            structure.append(line)

    # Save the last structure
    if structure:
        with open(f's{file_count}ptmsp2.xyz', 'w') as out_file:
            out_file.writelines(structure)



# Replace 'your_file.xyz' with the path to your XYZ file
split_xyz_file('xyz.format')
# Replace 'yourfile.dup' with your actual file name
lattice_lengths = extract_lattice_lengths('dump.lammpstrj')

# Print or process the lattice lengths
for i, lengths in enumerate(lattice_lengths, 1):

    # Read the XYZ file
    atoms = read(f's{i}ptmsp2.xyz')
    # Define lattice vectors for non-cubic lattice
    a_length = lengths[0]  # Length along x-axis in Angstroms
    b_length = lengths[1]  # Length along y-axis in Angstroms
    c_length = lengths[2]  # Length along z-axis in Angstroms

    lattice_vectors = np.array([[a_length, 0, 0],
                            [0, b_length, 0],
                            [0, 0, c_length]])

    # Set the lattice (cell)
    atoms.set_cell(lattice_vectors)

    # Ensure periodic boundary conditions are set if it's a bulk structure
    atoms.set_pbc([True, True, True])

    # Write to a VASP POSCAR file
    write(f's{i}ptmsp2.cif', atoms)
